In [12]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

In [26]:
import pymysql
import requests
"""
db = pymysql.connect(
    user='jisoo',
    passwd='53392c',
    db='caps_data',
    host='165.194.104.23',
    port=6689,
    charset='UTF8'
)
"""

db = pymysql.connect(
    user='root',
    passwd='dlatjsgh99',
    db='caps_data',
    host='localhost',
    port=3306,
    charset='UTF8'
)

cursor = db.cursor(pymysql.cursors.DictCursor)
select_query = "SELECT contentid, overview_kor FROM overview_tb"
cursor.execute(select_query)
results = cursor.fetchall()



In [8]:
from konlpy.tag import Okt  
okt=Okt()  
print(okt.morphs("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print(okt.nouns("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))  

['열심히', '코딩', '한', '당신', ',', '연휴', '에는', '여행', '을', '가봐요']
['코딩', '당신', '연휴', '여행']


In [5]:
#from konlpy.tag import Okt
from konlpy.tag import Kkma  

from nltk.tag import pos_tag
import numpy as np
from bs4 import BeautifulSoup

#okt=Okt()
kkma=Kkma()

for result in results:
    contentid = result['contentid']
    print(contentid)
    #BeautifulSoup(result['overview_kor'], 'html.parser').text
    noun_list = kkma.nouns(BeautifulSoup(result['overview_kor'], 'html.parser').text)
    print(noun_list)
    noun_list = list(set(noun_list))
    #print(noun_list)
    
    def removecheck(str):
        if(noun == '를' or noun == '과' or noun == '‘' or noun == '㎡' or noun.find('_') != -1 or noun.find('/') != -1 or noun.find('.') != -1 or noun.find('\'') != -1 or noun.find('㎢') != -1 or noun.find('\%') != -1) :
            return True
        if(any(chr.isdigit() for chr in str)) :
            return True
    
    temp_list = []
    while noun_list:
        noun = noun_list.pop()
        if not removecheck(noun):
            temp_list.append(noun)
    while temp_list:
        noun_list.append(temp_list.pop())
    
    
    print(noun_list)
    
    #
    
    insert_query = "INSERT IGNORE INTO mapping_tb(contentid, wordcount) VALUES(" + str(contentid) + ", " + str(len(noun_list)) +")"
    print(insert_query) 
    ####
    cursor.execute(insert_query)
    db.commit()
    print("content id and word count inserted")
    
    
    
    i = 0
    for noun in noun_list:
        i = i + 1
        #if(i > 30):
        #else if(i <= 30) :
        
        exits_query = "SELECT EXISTS (SELECT * FROM word_tb where word_kor = \'" + noun + "\') as success"
        cursor.execute(exits_query)
        exist_result = cursor.fetchone()
        print("exist result = " + str(exist_result['success']))
        
        if exist_result['success'] == 1 :
            select_query = "SELECT id, place_count from word_tb where word_kor=\'" + noun + "\'"
            cursor.execute(select_query)
            selected_word = cursor.fetchone()
            print("selected word id = " + str(selected_word['id']))
            print("selected word place count = " + str(selected_word['place_count']))
            print("korean word id selected")
            update_query = "UPDATE word_tb SET place_count = " + str(selected_word['place_count'] + 1) + " WHERE id = " + str(selected_word['id'])
            cursor.execute(update_query)
            db.commit()
            if selected_word['place_count'] + 1 <= 50 :
                update_query = "UPDATE word_tb SET place" + str(selected_word['place_count'] + 1) + " = " + str(contentid) + " WHERE id = " + str(selected_word['id'])
                cursor.execute(update_query)
                db.commit()
        else :
            insert_query = "INSERT IGNORE INTO word_tb(word_kor) VALUES(\'" + noun + "\')" #unique set 
            cursor.execute(insert_query)
            db.commit()
            print("korean word inserted")
        
            select_query = "SELECT id, place_count from word_tb where word_kor=\'" + noun + "\'"
            cursor.execute(select_query)
            selected_word = cursor.fetchone()
            print("selected word id = " + str(selected_word['id']))
            print("selected word place count = " + str(selected_word['place_count']))
            print("korean word id selected")
            update_query = "UPDATE word_tb SET place_count = 1 WHERE id = " + str(selected_word['id'])
            cursor.execute(update_query)
            db.commit()
            update_query = "UPDATE word_tb SET place1 = " + str(contentid) + " WHERE id = " + str(selected_word['id'])
            cursor.execute(update_query)
            db.commit()
        


264337
['1395', '1395년', '년', '경복궁', '인근', '창덕궁', '동궁', '과', '경희궁', '궁', '비교', '때', '북쪽', '위치', '때문', '북', '북궁', '5', '개', '궁전', '중', '임진', '전쟁', '1592', '1598', '화재', '건물', '소실', '이후', '고종', '1852', '1919', '시대', '흥', '대원군', '지도', '아래', '모든', '궁궐', '복원', '조선', '대표', '건축물', '경회루', '향', '향원정', '원정', '주변', '연못', '근정', '근정전', '전', '월대', '조각상', '당대', '예술', '스타일', '국립', '고궁', '박물관', '흥례문', '례', '문', '남쪽', '민속', '동쪽']
['중', '아래', '건축물', '대원군', '전', '고궁', '흥례문', '건물', '민속', '고종', '동쪽', '개', '전쟁', '조각상', '국립', '모든', '원정', '궁궐', '예술', '례', '흥', '이후', '근정전', '동궁', '때', '궁전', '지도', '위치', '북', '향원정', '북궁', '대표', '스타일', '연못', '조선', '문', '비교', '년', '때문', '인근', '창덕궁', '박물관', '궁', '임진', '월대', '당대', '화재', '경복궁', '시대', '경회루', '소실', '북쪽', '주변', '복원', '남쪽', '근정', '경희궁', '향']
INSERT IGNORE INTO mapping_tb(contentid, wordcount) VALUES(264337, 58)
content id and word count inserted
exist result = 0
korean word inserted
selected word id = 1
selected word place count = 0
korean word id selected
exist result = 

['남산', '서울', '타워', '국내', '최초', '타워형', '형', '관광', '명소', '탑', '꼭대기', '243', '과', '자체', '높이', '236.7', '포함', '해발', '약', '480', '아시아', '중', '최근', '40', '40년', '년', '만', '개장', '플라자', '2005', '2005년', '푸드', '이', '운영', '구성', '1969', '1969년', '라디오', '신호', '송출', '방송', '처음', '설립', '이제', '대표', '랜드', '마크', '마크이자', '이자', '다문화', '자리', '매', '매김', '김']
['중', '이', '아시아', '형', '자리', '포함', '라디오', '운영', '송출', '해발', '최근', '이제', '매김', '최초', '랜드', '관광', '이자', '국내', '설립', '처음', '자체', '다문화', '대표', '탑', '신호', '년', '서울', '남산', '개장', '구성', '마크이자', '매', '푸드', '만', '플라자', '타워', '마크', '김', '방송', '명소', '타워형', '높이', '꼭대기', '약']
INSERT IGNORE INTO mapping_tb(contentid, wordcount) VALUES(264550, 44)
content id and word count inserted
exist result = 1
selected word id = 1
selected word place count = 1
korean word id selected
exist result = 0
korean word inserted
selected word id = 119
selected word place count = 0
korean word id selected
exist result = 0
korean word inserted
selected word id = 120
selected word place cou

['도심', '한복판', '위치', '인사', '인사동', '동', '귀중', '전통', '상품', '전시', '중요', '장소', '골목길', '양쪽', '하나', '골목', '안', '갤러리', '레스토랑', '찻집', '카페', '심장', '이', '지역', '회화', '조각', '한국', '미술', '예시', '제공', '약', '100', '100개', '개', '유명', '민속', '예술', '중심지', '역할', '학', '학고재', '고재', '유망', '작가', '육성', '가나', '아트', '센터', '완벽', '보완', '보물', '인사', '인사동의', '동의', '가게', '개성', '연령층', '인기', '토요일', '14', '00', '22', '일요일', '10', '메인', '스트리트', '차량', '통행', '차단', '문화', '공간', '상점', '밖', '부스', '설치', '사탕', '상인', '점쟁이', '가판대', '수', '전통적', '공연', '외국인', '관광객', '체험', '미술품', '구입', '곳']
['미술', '인사동', '인기', '상점', '회화', '센터', '곳', '예술', '일요일', '차량', '구입', '조각', '제공', '공연', '위치', '심장', '완벽', '점쟁이', '통행', '중요', '문화', '하나', '부스', '약', '한복판', '메인', '양쪽', '갤러리', '개', '안', '사탕', '상인', '스트리트', '학', '작가', '도심', '상품', '예시', '역할', '전시', '아트', '카페', '개성', '이', '가판대', '민속', '장소', '관광객', '인사동의', '유망', '동', '찻집', '지역', '골목길', '인사', '육성', '한국', '고재', '외국인', '귀중', '가나', '수', '공간', '체험', '유명', '동의', '전통적', '학고재', '연령층', '가게', '골목', '토요일', '전통', '보물', '보

['창덕궁', '1405', '1405년', '년', '경복궁', '건축', '후', '번째', '궁궐', '조선', '시대', '왕', '본', '본궁', '궁', '5', '5개', '개', '궁중', '중', '보존', '궁전', '부지', '공공', '구역', '왕실', '거주', '건물', '뒷', '정원', '구성', '안식처', '수령', '300', '300년', '거대', '나무', '연못', '정자', '9', '9대', '대', '성종', '때', '거주지', '사용', '시작', '중요성', '1592', '1592년', '일본', '한국', '침략', '당시', '화난', '시민', '불', '광해군', '덕분', '1611', '1611년', '복원', '오늘날', '인정전', '대전', '선정전', '낙선재', '등', '문화재', '소장', '안당', '뒤', '비밀', '태종', '건립', '왕족', '역할', '예전', '부원', '금원', '고종', '집권', '비원', '이름', '가능', '한', '자연', '유지', '절대적', '필요', '사람', '손', '부용', '부용정', '정', '부', '부용지', '용지', '주', '함루', '어수', '어수문', '문', '영화당', '불로', '불로문', '애', '애연정', '연정', '당', '차지', '볼거리', '하나', '수', '시기', '단풍', '절정', '잎', '가을', '수세기', '동안', '한국인', '1997', '1997년', '12', '12월', '월', '이탈리아', '나폴리', '열린', '위원회', '회의', '유', '유네스코', '네스', '코', '세계', '문화', '유산', '유산위원회', '지정']
['중', '잎', '불', '유네스코', '비밀', '가능', '불로문', '월', '당', '한', '사용', '안식처', '손', '후', '조선', '문화', '회의', '애연정', '년', '중요성', '하나', '대전

['명동', '서울', '주요', '쇼핑', '쇼핑가', '중', '지하철', '4', '4호선', '호', '선', '명동역과', '역과', '을지', '롯데', '백화점', '시작', '개', '메인', '스트리트', '블록', '중앙', '브랜드', '상점', '거리', '골목', '판매', '일반적', '제품', '옷', '신발', '액세서리', '포함', '남대', '남대문', '문', '동대문', '디자이너', '신세계', '레', '눈', '스퀘어', '엠', '플라자', '등', '이곳', '지점', '합리적', '가격', '프리미엄', '라벨', '기타', '패션', '상품', '패밀리', '레스토랑', '패스트', '푸드', '한식', '양식', '일식', '식당', '돈까스', '칼국수', '전문', '이', '지역', '사업', '미용실', '은행', '극장']
['중', '쇼핑가', '이', '동대문', '남대', '신세계', '양식', '한식', '상점', '패밀리', '포함', '개', '남대문', '합리적', '브랜드', '극장', '골목', '지하철', '스트리트', '거리', '액세서리', '지점', '지역', '일반적', '선', '엠', '등', '주요', '중앙', '프리미엄', '롯데', '은행', '돈까스', '라벨', '제품', '블록', '명동', '레', '명동역과', '역과', '칼국수', '전문', '레스토랑', '식당', '판매', '패스트', '일식', '문', '시작', '디자이너', '미용실', '사업', '서울', '스퀘어', '패션', '쇼핑', '상품', '이곳', '푸드', '플라자', '을지', '백화점', '가격', '옷', '호', '눈', '기타', '신발', '메인']
INSERT IGNORE INTO mapping_tb(contentid, wordcount) VALUES(264312, 69)
content id and word count inserted
exist result = 1
s

...

exist result = 0
korean word inserted
selected word id = 5285
selected word place count = 0
korean word id selected
exist result = 1
selected word id = 531
selected word place count = 70
korean word id selected
exist result = 1
selected word id = 2160
selected word place count = 11
korean word id selected
exist result = 0
korean word inserted
selected word id = 5286
selected word place count = 0
korean word id selected
exist result = 1
selected word id = 618
selected word place count = 5
korean word id selected
exist result = 1
selected word id = 4834
selected word place count = 1
korean word id selected
exist result = 1
selected word id = 12
selected word place count = 97
korean word id selected
exist result = 1
selected word id = 677
selected word place count = 12
korean word id selected
exist result = 1
selected word id = 5253
selected word place count = 1
korean word id selected
exist result = 1
selected word id = 965
selected word place count = 8
korean word id selected
exist resu

selected word id = 38
selected word place count = 190
korean word id selected
exist result = 1
selected word id = 278
selected word place count = 5
korean word id selected
exist result = 1
selected word id = 2914
selected word place count = 1
korean word id selected
exist result = 0
korean word inserted
selected word id = 5300
selected word place count = 0
korean word id selected
exist result = 1
selected word id = 615
selected word place count = 19
korean word id selected
exist result = 1
selected word id = 767
selected word place count = 36
korean word id selected
exist result = 0
korean word inserted
selected word id = 5301
selected word place count = 0
korean word id selected
exist result = 1
selected word id = 248
selected word place count = 14
korean word id selected
exist result = 0
korean word inserted
selected word id = 5302
selected word place count = 0
korean word id selected
exist result = 0
korean word inserted
selected word id = 5303
selected word place count = 0
korean w

exist result = 0
korean word inserted
selected word id = 5329
selected word place count = 0
korean word id selected
1057049
['야경', '유명', '용', '용연', '연', '연못', '바닷물', '전', '산', '골짜기', '끝', '담수', '형성', '바닥', '정자', '목조', '구름', '다리', '계곡', '풍경', '조화', '전설', '하늘', '빗물', '용의', '의미', '이름', '서쪽', '200', '미터', '용두암', '믿음']
['의미', '산', '바닷물', '유명', '끝', '바닥', '용의', '서쪽', '빗물', '담수', '믿음', '다리', '야경', '목조', '전설', '구름', '풍경', '용두암', '계곡', '조화', '연못', '형성', '용', '골짜기', '하늘', '용연', '정자', '미터', '이름', '연', '전']
INSERT IGNORE INTO mapping_tb(contentid, wordcount) VALUES(1057049, 31)
content id and word count inserted
exist result = 1
selected word id = 987
selected word place count = 23
korean word id selected
exist result = 1
selected word id = 1125
selected word place count = 58
korean word id selected
exist result = 1
selected word id = 1938
selected word place count = 8
korean word id selected
exist result = 1
selected word id = 233
selected word place count = 69
korean word id selected
exist resul

['백록담', '제주', '제주도', '도', '중심', '한라산', '정상', '위치', '화산', '화산작용', '작용', '형성', '호수', '둘레', '3', '직경', '500', '약', '360', '개', '기생', '기생화산', '방문객', '구름', '느낌', '수']
['중심', '기생화산', '도', '개', '느낌', '화산', '백록담', '제주도', '정상', '화산작용', '기생', '구름', '위치', '작용', '직경', '호수', '형성', '제주', '수', '둘레', '한라산', '방문객', '약']
INSERT IGNORE INTO mapping_tb(contentid, wordcount) VALUES(264591, 23)
content id and word count inserted
exist result = 1
selected word id = 488
selected word place count = 19
korean word id selected
exist result = 1
selected word id = 5055
selected word place count = 1
korean word id selected
exist result = 1
selected word id = 531
selected word place count = 73
korean word id selected
exist result = 1
selected word id = 12
selected word place count = 100
korean word id selected
exist result = 1
selected word id = 1548
selected word place count = 3
korean word id selected
exist result = 1
selected word id = 3833
selected word place count = 5
korean word id selected
exist result = 0
ko

exist result = 1
selected word id = 679
selected word place count = 29
korean word id selected
exist result = 1
selected word id = 1455
selected word place count = 10
korean word id selected
exist result = 1
selected word id = 2146
selected word place count = 22
korean word id selected
exist result = 1
selected word id = 4825
selected word place count = 16
korean word id selected
exist result = 1
selected word id = 1249
selected word place count = 13
korean word id selected
exist result = 1
selected word id = 224
selected word place count = 93
korean word id selected
exist result = 1
selected word id = 2710
selected word place count = 16
korean word id selected
exist result = 1
selected word id = 2128
selected word place count = 16
korean word id selected
exist result = 1
selected word id = 5165
selected word place count = 3
korean word id selected
exist result = 1
selected word id = 1872
selected word place count = 2
korean word id selected
exist result = 1
selected word id = 1784
sel

In [24]:
#select words

select_query = "SELECT * FROM word_tb WHERE ignore_flag_foreign = 0 AND ignore_flag_specific = 0"
cursor.execute(select_query)
results = cursor.fetchall()

for result in results:
    word_id = str(result['id'])
    word_kor = result['word_kor']
    place_count = str(result['place_count'])
    place1 = str(result['place1'])
    place2 = str(result['place2'])
    place3 = str(result['place3'])
    place4 = str(result['place4'])
    place5 = str(result['place5'])
    place6 = str(result['place6'])
    place7 = str(result['place7'])
    place8 = str(result['place8'])
    place9 = str(result['place9'])
    place10 = str(result['place10'])
    place11 = str(result['place11'])
    place12 = str(result['place12'])
    place13 = str(result['place13'])
    place14 = str(result['place14'])
    place15 = str(result['place15'])
    place16 = str(result['place16'])
    place17 = str(result['place17'])
    place18 = str(result['place18'])
    place19 = str(result['place19'])
    place20 = str(result['place20'])
    place21 = str(result['place21'])
    place22 = str(result['place22'])
    place23 = str(result['place23'])
    place24 = str(result['place24'])
    place25 = str(result['place25'])
    place26 = str(result['place26'])
    place27 = str(result['place27'])
    place28 = str(result['place28'])
    place29 = str(result['place29'])
    place30 = str(result['place30'])
    place31 = str(result['place31'])
    place32 = str(result['place32'])
    place33 = str(result['place33'])
    place34 = str(result['place34'])
    place35 = str(result['place35'])
    place36 = str(result['place36'])
    place37 = str(result['place37'])
    place38 = str(result['place38'])
    place39 = str(result['place39'])
    place40 = str(result['place40'])
    place41 = str(result['place41'])
    place42 = str(result['place42'])
    place43 = str(result['place43'])
    place44 = str(result['place44'])
    place45 = str(result['place45'])
    place46 = str(result['place46'])
    place47 = str(result['place47'])
    place48 = str(result['place48'])
    place49 = str(result['place49'])
    place50 = str(result['place50'])
    
    #insert_query = "INSERT INTO word2_tb(word_kor, place_count, place1, place2, place3, place4, place5, place6, place7, place8, place9, place10, place11, place12, place13, place14, place15, place16, place17, place18, place19, place20, place21, place22, place23, place24, place25, place26, place27, place28, place29, place30, place31, place32, place33, place34, place35, place36, place37, place38, place39, place40, place41, place42, place43, place44, place45, place46, place47, place48, place49, place50) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    insert_query = "INSERT INTO word3_tb(word_id, word_kor, place_count) VALUES(%s, %s, %s)"
    #values = (word_kor, place_count, place1, place2, place3, place4, place5, place6, place7, place8, place9, place10, place11, place12, place13, place14, place15, place16, place17, place18, place19, place20, place21, place22, place23, place24, place25, place26, place27, place28, place29, place30, place31, place32, place33, place34, place35, place36, place37, place38, place39, place40, place41, place42, place43, place44, place45, place46, place47, place48, place49, place50)
    values = (word_id, word_kor, place_count)
    cursor.execute(insert_query, values)
    db.commit()
    print("inserted")

inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
i

inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
i

In [28]:
select_query = "SELECT * FROM word_tb WHERE ignore_flag_foreign = 0 AND ignore_flag_specific = 0"
cursor.execute(select_query)
results = cursor.fetchall()
for result in results:
    print(result['word_kor'])
    place_count = result['place_count']
    
    if place_count >= 50 :
        for i in range(50):
            #i + 1
            key = "place" + str(i+1)
            update_query = "UPDATE word3_tb SET place" + str(i+1) + " = " + str(result[key]) + " WHERE word_id = " + str(result['id'])
            cursor.execute(update_query)
            db.commit()
            
    elif place_count < 50 :
        for i in range(place_count) :
            key = "place" + str(i+1)
            update_query = "UPDATE word3_tb SET place" + str(i+1) + " = " + str(result[key]) + " WHERE word_id = " + str(result['id'])
            cursor.execute(update_query)
            db.commit()
        
    
    


중
아래
건축물
고궁
건물
민속
고종
동쪽
전쟁
조각상
국립
모든
궁궐
예술
이후
때
궁전
지도
위치
대표
연못
조선
비교
인근
박물관
궁
임진
당대
화재
시대
소실
북쪽
주변
복원
남쪽
놀이
실내
역사
관광객
장소
모두
세계
놀이기구
계절
추가
운영
곳
다양
호숫가
야외
등
관광
최대
한국인
공연
하다
공원
한국
환상적인
편의
모형
완벽
최고급
규모
시설
경험
외국인
음식
각국
전시
포함
송출
해발
최근
이제
최초
국내
설립
처음
자체
다문화
탑
신호
서울
남산
개장
구성
방송
명소
높이
꼭대기
동네
오늘날
기회
마을
사람
실제
한옥
찻집
전통
존중
주요
제공
집
가옥
문화
종묘
이름
몰입
방문객
유래
미술
인기
상점
회화
일요일
차량
구입
조각
심장
점쟁이
통행
중요
하나
한복판
사탕
상인
작가
도심
상품
예시
역할
카페
개성
가판대
지역
골목길
공간
체험
유명
전통적
연령층
가게
골목
토요일
보물
보완
설치
차단
중심지
밖
미술품
한강
유리
전망
층
지점
자랑
일몰
고층
총
도시
잎
불
유네스코
비밀
가능
사용
안식처
손
중요성
집권
예전
필요
자연
태종
문화재
수령
왕
공공
유산
정원
광해군
낙선재
소장
본
수세기
당시
덕분
거주
열린
성종
보존
위원회
거대
거주지
시기
구역
유지
단풍
왕족
정자
절대적
건축
궁중
뒷
침략
시민
일본
부지
지정
차지
왕실
뒤
코
나무
번째
시작
가을
절정
동안
볼거리
건립
신세계
양식
한식
남대문
합리적
극장
지하철
거리
일반적
중앙
은행
돈까스
제품
칼국수
전문
식당
판매
일식
미용실
사업
백화점
가격
옷
기타
신발
연상
음악
대교
생태
공중
지면
재생
회
빛
관찰
밤
반포
강
낮
하루
연출
조명
물
무대
무지개
분수
광장
한약
묘사
재건
맛
소유자
한글
원래
윷놀이
예정
태권도
분위기
계급
수도
활동
사회적
재료
생활
정문
참여
예절
주목
여유
소유
전시관
숲
형태
교육
환경
다도
귀족
성격
복제
앞
현지인
기념
신중
고택
한지
공무원
공개
중심
왕조
시범
평화
제외
분주
산책
쓰기
어린이
한복
학교
이해
낭만
지하

In [19]:

#request_url = "https://openapi.naver.com/v1/papago/n2mt"
#headers = {"X-Naver-Client-Id": "mTQMl8hMG1noD5RSnfEP", "X-Naver-Client-Secret": "VfehlrZ47y"}
#headers = {"X-Naver-Client-Id": "82Vth9owGFWuENapJlzq", "X-Naver-Client-Secret": "aWD_4ySZbz"}
#headers =  {"X-Naver-Client-Id": "Q61xGMjA2mjiyoJGT4U8", "X-Naver-Client-Secret": "anRzOdP2HG"}

def translate_text(target, text):
    
    import six
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    result = translate_client.translate(text, target_language=target)
    return result["translatedText"]
   


select_query = "SELECT id, word_eng FROM word_tb where word_kor is NULL"
cursor.execute(select_query)
results = cursor.fetchall()

for result in results:
    id = result['id']
    word_eng = result['word_eng']
    
    word_kor = translate_text("ko", word_eng)
    
    #params = {"source": "en", "target": "ko", "text": word_eng}
    #response = requests.post(request_url, headers=headers, data=params)
    #kor_result = response.json()
    #print("korean word : " + kor_result['message']['result']['translatedText'])
        
    update_query = "UPDATE word_tb SET word_kor = %s WHERE id=" + str(id)
    #value = kor_result['message']['result']['translatedText']
    value = word_kor
    
    cursor.execute(update_query, value)
    db.commit()
    print("~one result end~")

    
print("~all results end~")

~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result en

~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result end~
~one result en

KeyboardInterrupt: 

In [15]:
url = "https://opendict.korean.go.kr/api/search"
headers = {"key":"3F4750E75197DCFD2C897BF0780E9160"}
word = '나무'
word = word.encode('utf-8')
params = {"target_type":"search", "part":"word", "sort":"dict", "start":"1", "num":"1", "q":word}
response = requests.post(url, headers=headers, data=params)
print(response)

#meaning_kor = result['definition']

<Response [200]>


In [6]:


    
#translate_text("ko","hello apple bubble")

Text: hello apple bubble
Translation: 안녕 사과 거품
Detected source language: en


In [7]:
import requests

request_url = "https://openapi.naver.com/v1/search/image"
headers = {"X-Naver-Client-Id": "3rnOcTv2HxD_zj8BqHMW", "X-Naver-Client-Secret": "EC6tcZi5fC"}
params = {"query": "%EC%A3%BC%EC%8B%9D"}
response = requests.post(request_url, headers=headers, data=params)
image_result = response.json()
print(image_result)
#image_link = image_result['message']['result']['link']

{'errorMessage': '/v1/search/image : Bad Request. (Check Extension or Required Parameter)', 'errorCode': '400'}
